# 追踪可视化

可视化视频中的框的颜色的解释:
- 红色: 真值。
- 蓝色: 原模型预测。
- 灰色: 纠偏过程中tracker的预测结果。
- 黄色: 纠偏后的结果, 如果未发生纠偏, 与灰色框相同(会将灰色完全覆盖)。
- 橙色: tracker对应的search图像。
- 紫色: 纠偏网络对应的search图像。
- 黑色: template对应的search图像。
- 绿色+: 眼动cx和cy。

In [1]:
import os
import cv2
import numpy as np
import glob
from tqdm import tqdm
import torch
import pandas as pd
import pickle

### NFS

In [3]:
runid = '74'
subject_name = 'liweilong'
fps = 240 # TODO 30 240
# data
# with open(f'/data/xgxy/BeiJing/code/VideoX/SeqTrack/workspace/plot/lasot/eval_data_run{runid.zfill(2)}.pkl', 'rb') as f:
#     eval_data_exp = pickle.load(f)
# sample_list = eval_data_exp['sequences']
# 0%-10%
sequence_list = ['pingpong_2']
# 0%-20%
sequence_list = ['Skydiving', 'bowling_1', 'pingpong_2']
# 单独跑一些样本
# sequence_list = ['airboard_1', 'basketball_1', 'car', 'dog', 'shuffleboard_1', 'soccer_ball']
sample_list = sequence_list
print('样本数量：', len(sample_list))

output_dir = os.path.join('/data/guohua/BeiJing/code/VideoX.back/visual', 'nfs', subject_name, runid)
ori_pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_999/nfs' 
seqtrack_pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/nfs' # 修改成纠偏网络的输入
pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/nfs'
search_bbox_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/nfs'
search_bbox_eye_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/nfs'
template_bbox_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/nfs'
eye_dir = f'/data/guohua/BeiJing/data/NFS-eye-sy/{subject_name}'
data_dir = '/data/guohua/BeiJing/data/Nfs' # /data/guohua/BeiJing/data/lasot

# 字体参数
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
line_type = 2

video_dir = os.path.join(output_dir)
os.makedirs(video_dir, exist_ok=True)

for s in sample_list:
    # if s in sequence_list: continue
    s_dir = s # lasot .split('-')[0]
    sample_dir = os.path.join(data_dir, s_dir, '240') # 240 
    image_list = sorted(glob.glob(f"{os.path.join(sample_dir, s)}/*.jpg"))
    # 真值
    gt_file_path = os.path.join(sample_dir, 'groundtruth.txt')
    gt_list = np.loadtxt(gt_file_path, delimiter=',', dtype=np.int32)
    # 原模型预测值
    ori_pred_file_path = os.path.join(ori_pred_dir, f'nfs_{s}.txt')
    ori_pred_list = np.loadtxt(ori_pred_file_path, delimiter='\t', dtype=np.int32)
    # 纠偏网络的输入
    seqtrack_pred_file_path = os.path.join(seqtrack_pred_dir, f'nfs_{s}_seqtrack.txt')
    seqtrack_pred_list = np.loadtxt(seqtrack_pred_file_path, delimiter='\t', dtype=np.int32)
    # 预测值
    pred_file_path = os.path.join(pred_dir, f'nfs_{s}.txt')
    pred_list = np.loadtxt(pred_file_path, delimiter='\t', dtype=np.int32)
    # search_bbox
    search_bbox_file_path = os.path.join(search_bbox_dir, f'nfs_{s}_search.txt')
    search_bbox_list = np.loadtxt(search_bbox_file_path, delimiter='\t', dtype=np.int32)
    # search_bbox_eye
    search_bbox_eye_file_path = os.path.join(search_bbox_eye_dir, f'nfs_{s}_search_eye.txt')
    search_bbox_eye_list = np.loadtxt(search_bbox_eye_file_path, delimiter='\t', dtype=np.int32)
    # template_bbox
    template_bbox_file_path = os.path.join(template_bbox_dir, f'nfs_{s}_template.txt')
    template_bbox_list = np.loadtxt(template_bbox_file_path, delimiter='\t', dtype=np.int32)
    # 眼动数据
    eye_file_path = os.path.join(eye_dir, s_dir, f'{s}.txt')
    eye_list = np.loadtxt(eye_file_path, delimiter=',', dtype=np.int32)

    # 获取图片尺寸
    first_image = cv2.imread(image_list[0])
    height, width, layers = first_image.shape
    # 初始化视频写入对象
    video_path = os.path.join(video_dir, f'{s}.mp4')
    # if os.path.exists(video_path):
    #     continue
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(video_path, fourcc, fps=fps, frameSize=(width, height))
    
    gtw, gth = gt_list[0][2], gt_list[0][3]
    # 遍历每张图片并绘制真值和预测框
    for j, image_path in enumerate(tqdm(image_list, desc=f'{s}')):
        img = cv2.imread(image_path)
        
        # 获取真值, 预测结果和眼动结果
        gt = gt_list[j]
        ori_pred = ori_pred_list[j]
        seqtrack_pred = seqtrack_pred_list[j]
        pred = pred_list[j]
        search_bbox = search_bbox_list[j]
        search_bbox_eye = search_bbox_eye_list[j]
        eye = eye_list[j]
        template_bbox = template_bbox_list[j]
        
        if gt is not None:
            # 计算真值框的左上角和右下角坐标
            gt_x, gt_y, gt_w, gt_h = int(gt[0]), int(gt[1]), int(gt[2]), int(gt[3])
            gt_top_left = (gt_x, gt_y)
            gt_bottom_right = (gt_x + gt_w, gt_y + gt_h)
            # 画真值框
            cv2.rectangle(img, gt_top_left, gt_bottom_right, (0, 0, 255), 2)  # 红色框
            # text_org = (gt_top_left[0], gt_top_left[1] + gt[3] - 12)
            # cv2.putText(img, f'Ground-truth', text_org, font, font_scale, (0, 0, 255), line_type)
            
        if ori_pred is not None:
            # 计算预测框的左上角和右下角坐标
            ori_pred_x, ori_pred_y, ori_pred_w, ori_pred_h = int(ori_pred[0]), int(ori_pred[1]), int(ori_pred[2]), int(ori_pred[3])
            ori_pred_top_left = (ori_pred_x, ori_pred_y)
            ori_pred_bottom_right = (ori_pred_x + ori_pred_w, ori_pred_y + ori_pred_h)
            # 画预测框
            cv2.rectangle(img, ori_pred_top_left, ori_pred_bottom_right, (255, 0, 0), 2)  # 蓝色框
            # 标记预测值
            # text_org = (ori_pred_top_left[0] - 15, ori_pred_top_left[1] - 10)
            # cv2.putText(img, f'{frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (255, 0, 0), line_type)
            # cv2.putText(img, f'Original prediction', text_org, font, font_scale, (255, 0, 0), line_type)
        
        if seqtrack_pred is not None:
            # 计算预测框的左上角和右下角坐标
            seqtrack_pred_x, seqtrack_pred_y, seqtrack_pred_w, seqtrack_pred_h = int(seqtrack_pred[0]), int(seqtrack_pred[1]), int(seqtrack_pred[2]), int(seqtrack_pred[3])
            seqtrack_pred_top_left = (seqtrack_pred_x, seqtrack_pred_y)
            seqtrack_pred_bottom_right = (seqtrack_pred_x + seqtrack_pred_w, seqtrack_pred_y + seqtrack_pred_h)
            # 画预测框
            cv2.rectangle(img, seqtrack_pred_top_left, seqtrack_pred_bottom_right, (181, 181, 181), 2) # grey71

        if pred is not None:
            # 计算预测框的左上角和右下角坐标
            pred_x, pred_y, pred_w, pred_h = int(pred[0]), int(pred[1]), int(pred[2]), int(pred[3])
            pred_top_left = (pred_x, pred_y)
            pred_bottom_right = (pred_x + pred_w, pred_y + pred_h)
            # 画预测框
            cv2.rectangle(img, pred_top_left, pred_bottom_right, (0, 255, 255), 2)  # 黄
            # 标记预测值
            # text_org = (pred_top_left[0] - 10, pred_top_left[1] - 12)
            # cv2.putText(img, f'{frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (255, 0, 0), line_type)
            # cv2.putText(img, f'Improved prediction', text_org, font, font_scale, (0, 255, 255), line_type)

        if search_bbox is not None:
            # 计算预测框的左上角和右下角坐标
            search_bbox_x0, search_bbox_y0, search_bbox_x1, search_bbox_y1 = int(search_bbox[0]), int(search_bbox[1]), int(search_bbox[2]), int(search_bbox[3])
            # 画预测框
            cv2.rectangle(img, [search_bbox_x0, search_bbox_y0], [search_bbox_x1, search_bbox_y1], (0, 165, 255), 2)  # Yellow1
        
        if search_bbox_eye is not None:
            # 计算预测框的左上角和右下角坐标
            search_bbox_eye_x0, search_bbox_eye_y0, search_bbox_eye_x1, search_bbox_eye_y1 = int(search_bbox_eye[0]), int(search_bbox_eye[1]), int(search_bbox_eye[2]), int(search_bbox_eye[3])
            # 画预测框
            cv2.rectangle(img, [search_bbox_eye_x0, search_bbox_eye_y0], [search_bbox_eye_x1, search_bbox_eye_y1], (255, 0, 255), 2)  # 紫色
        
        if template_bbox is not None:
            # 计算预测框的左上角和右下角坐标
            template_bbox_x0, template_bbox_y0, template_bbox_x1, template_bbox_y1 = int(template_bbox[0]), int(template_bbox[1]), int(template_bbox[2]), int(template_bbox[3])
            if j == 0: 
                template_top_left = (template_bbox_x0, template_bbox_y0)
                template_bottom_right = (template_bbox_x0 + template_bbox_x1, template_bbox_y0 + template_bbox_y1)
                # 画预测框
                cv2.rectangle(img, template_top_left, template_bottom_right, (0, 0, 0), 2)  # black
            else:
                cv2.rectangle(img, [template_bbox_x0, template_bbox_y0], [template_bbox_x1, template_bbox_y1], (0, 0, 0), 2) # black

        if eye is not None:
            # 计算预测框的左上角和右下角坐标
            # eye_x, eye_y, eye_w, eye_h = int(eye[0]), int(eye[1]), int(eye[2]), int(eye[3])
            # eye_top_left = (eye_x - eye_w//2, eye_y - eye_h//2)
            # eye_bottom_right = (eye_x + eye_w//2, eye_y + eye_h//2)
            # 画预测框
            # cv2.rectangle(img, eye_top_left, eye_bottom_right, (0, 255, 0), 2)  # 绿色框
            # 标记预测值
            # text_org = (eye_top_left[0], eye_top_left[1] - 10)
            # cv2.putText(img, f'{eye_frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (0, 255, 0), line_type)
            eye_x, eye_y= int(eye[0]), int(eye[1])
            cv2.drawMarker(img, (eye_x, eye_y), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2)  # 绿色"+"

        # 写入视频
        video.write(img)

    # 释放视频写入对象
    video.release()
    cv2.destroyAllWindows()

样本数量： 3


Skydiving:   0%|          | 6/1568 [00:00<00:27, 56.37it/s]

pingpong_2: 100%|██████████| 10209/10209 [01:26<00:00, 118.17it/s]


### UAV

In [4]:
runid = '74'
subject_name = 'liweilong'
fps = 30 # TODO 30 240
# data
# with open(f'/data/xgxy/BeiJing/code/VideoX/SeqTrack/workspace/plot/lasot/eval_data_run{runid.zfill(2)}.pkl', 'rb') as f:
#     eval_data_exp = pickle.load(f)
# sample_list = eval_data_exp['sequences']
# 0%-10%
sequence_list = ['uav4', 'uav8']
# 0%-20%
sequence_list = ['bike2', 'bird1_2', 'bird1_3', 'uav1_3', 'uav4', 'uav7', 'uav8']
# 单独跑一些样本
# sequence_list = ['bike1', 'bird1_1', 'car15', 'person10', 'uav5', 'wakeboard10', 'truck1', 'building5', 'boat5'] # 'bike1', 'car15', 'person10', 'uav5', 'wakeboard10', 'truck1', 'building5', 'boat5'
sample_list = sequence_list
print('样本数量：', len(sample_list))

output_dir = os.path.join('/data/guohua/BeiJing/code/VideoX.back/visual', 'uav', subject_name, runid)
ori_pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_999/uav' 
seqtrack_pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/uav' # 修改成纠偏网络的输入
pred_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/uav'
search_bbox_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/uav'
search_bbox_eye_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/uav'
template_bbox_dir = f'/data/guohua/BeiJing/code/VideoX.back/SeqTrack/test/tracking_results/seqtrack/seqtrack_b256_{runid.zfill(3)}/uav'
eye_dir = f'/data/guohua/BeiJing/data/UAV-eye-sy/{subject_name}'
new_eye_dir = '/data/guohua/BeiJing/data/lasot-eye-sy'
data_dir = '/data/guohua/BeiJing/data/UAV123/UAV_Lasot_struc' # /data/guohua/BeiJing/data/lasot

# 字体参数
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
line_type = 2

video_dir = os.path.join(output_dir)
os.makedirs(video_dir, exist_ok=True)

for s in sample_list:
    # if s in sequence_list: continue
    s_dir = s.split('_')[0]
    sample_dir = os.path.join(data_dir, s_dir, s) # lasot 
    image_list = sorted(glob.glob(f"{os.path.join(sample_dir, 'img')}/*.jpg"))
    # 真值
    gt_file_path = os.path.join(sample_dir, 'groundtruth.txt')
    gt_list = np.loadtxt(gt_file_path, delimiter=',', dtype=np.int32)
    # 原模型预测值
    ori_pred_file_path = os.path.join(ori_pred_dir, f'uav_{s}.txt')
    ori_pred_list = np.loadtxt(ori_pred_file_path, delimiter='\t', dtype=np.int32)
    # 纠偏网络的输入
    seqtrack_pred_file_path = os.path.join(seqtrack_pred_dir, f'uav_{s}_seqtrack.txt')
    seqtrack_pred_list = np.loadtxt(seqtrack_pred_file_path, delimiter='\t', dtype=np.int32)
    # 预测值
    pred_file_path = os.path.join(pred_dir, f'uav_{s}.txt')
    pred_list = np.loadtxt(pred_file_path, delimiter='\t', dtype=np.int32)
    # search_bbox
    search_bbox_file_path = os.path.join(search_bbox_dir, f'uav_{s}_search.txt')
    search_bbox_list = np.loadtxt(search_bbox_file_path, delimiter='\t', dtype=np.int32)
    # search_bbox_eye
    search_bbox_eye_file_path = os.path.join(search_bbox_eye_dir, f'uav_{s}_search_eye.txt')
    search_bbox_eye_list = np.loadtxt(search_bbox_eye_file_path, delimiter='\t', dtype=np.int32)
    # template_bbox
    template_bbox_file_path = os.path.join(template_bbox_dir, f'uav_{s}_template.txt')
    template_bbox_list = np.loadtxt(template_bbox_file_path, delimiter='\t', dtype=np.int32)
    # 眼动数据
    eye_file_path = os.path.join(eye_dir, s, f'{s}.txt')
    eye_list = np.loadtxt(eye_file_path, delimiter=',', dtype=np.int32)

    # 获取图片尺寸
    first_image = cv2.imread(image_list[0])
    height, width, layers = first_image.shape
    # 初始化视频写入对象
    video_path = os.path.join(video_dir, f'{s}.mp4')
    # if os.path.exists(video_path):
    #     continue
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(video_path, fourcc, fps=fps, frameSize=(width, height))
    
    gtw, gth = gt_list[0][2], gt_list[0][3]
    # 遍历每张图片并绘制真值和预测框
    for j, image_path in enumerate(tqdm(image_list, desc=f'{s}')):
        img = cv2.imread(image_path)
        
        # 获取真值, 预测结果和眼动结果
        gt = gt_list[j]
        ori_pred = ori_pred_list[j]
        seqtrack_pred = seqtrack_pred_list[j]
        pred = pred_list[j]
        search_bbox = search_bbox_list[j]
        search_bbox_eye = search_bbox_eye_list[j]
        eye = eye_list[j]
        template_bbox = template_bbox_list[j]
        
        if gt is not None:
            # 计算真值框的左上角和右下角坐标
            gt_x, gt_y, gt_w, gt_h = int(gt[0]), int(gt[1]), int(gt[2]), int(gt[3])
            gt_top_left = (gt_x, gt_y)
            gt_bottom_right = (gt_x + gt_w, gt_y + gt_h)
            # 画真值框
            cv2.rectangle(img, gt_top_left, gt_bottom_right, (0, 0, 255), 2)  # 红色框
            # text_org = (gt_top_left[0], gt_top_left[1] + gt[3] - 12)
            # cv2.putText(img, f'Ground-truth', text_org, font, font_scale, (0, 0, 255), line_type)
            
        if ori_pred is not None:
            # 计算预测框的左上角和右下角坐标
            ori_pred_x, ori_pred_y, ori_pred_w, ori_pred_h = int(ori_pred[0]), int(ori_pred[1]), int(ori_pred[2]), int(ori_pred[3])
            ori_pred_top_left = (ori_pred_x, ori_pred_y)
            ori_pred_bottom_right = (ori_pred_x + ori_pred_w, ori_pred_y + ori_pred_h)
            # 画预测框
            cv2.rectangle(img, ori_pred_top_left, ori_pred_bottom_right, (255, 0, 0), 2)  # 蓝色框
            # 标记预测值
            # text_org = (ori_pred_top_left[0] - 15, ori_pred_top_left[1] - 10)
            # cv2.putText(img, f'{frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (255, 0, 0), line_type)
            # cv2.putText(img, f'Original prediction', text_org, font, font_scale, (255, 0, 0), line_type)
        
        if seqtrack_pred is not None:
            # 计算预测框的左上角和右下角坐标
            seqtrack_pred_x, seqtrack_pred_y, seqtrack_pred_w, seqtrack_pred_h = int(seqtrack_pred[0]), int(seqtrack_pred[1]), int(seqtrack_pred[2]), int(seqtrack_pred[3])
            seqtrack_pred_top_left = (seqtrack_pred_x, seqtrack_pred_y)
            seqtrack_pred_bottom_right = (seqtrack_pred_x + seqtrack_pred_w, seqtrack_pred_y + seqtrack_pred_h)
            # 画预测框
            cv2.rectangle(img, seqtrack_pred_top_left, seqtrack_pred_bottom_right, (181, 181, 181), 2) # grey71

        if pred is not None:
            # 计算预测框的左上角和右下角坐标
            pred_x, pred_y, pred_w, pred_h = int(pred[0]), int(pred[1]), int(pred[2]), int(pred[3])
            pred_top_left = (pred_x, pred_y)
            pred_bottom_right = (pred_x + pred_w, pred_y + pred_h)
            # 画预测框
            cv2.rectangle(img, pred_top_left, pred_bottom_right, (0, 255, 255), 2)  # 黄
            # 标记预测值
            # text_org = (pred_top_left[0] - 10, pred_top_left[1] - 12)
            # cv2.putText(img, f'{frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (255, 0, 0), line_type)
            # cv2.putText(img, f'Improved prediction', text_org, font, font_scale, (0, 255, 255), line_type)

        if search_bbox is not None:
            # 计算预测框的左上角和右下角坐标
            search_bbox_x0, search_bbox_y0, search_bbox_x1, search_bbox_y1 = int(search_bbox[0]), int(search_bbox[1]), int(search_bbox[2]), int(search_bbox[3])
            # 画预测框
            cv2.rectangle(img, [search_bbox_x0, search_bbox_y0], [search_bbox_x1, search_bbox_y1], (0, 165, 255), 2)  # Yellow1
        
        if search_bbox_eye is not None:
            # 计算预测框的左上角和右下角坐标
            search_bbox_eye_x0, search_bbox_eye_y0, search_bbox_eye_x1, search_bbox_eye_y1 = int(search_bbox_eye[0]), int(search_bbox_eye[1]), int(search_bbox_eye[2]), int(search_bbox_eye[3])
            # 画预测框
            cv2.rectangle(img, [search_bbox_eye_x0, search_bbox_eye_y0], [search_bbox_eye_x1, search_bbox_eye_y1], (255, 0, 255), 2)  # 紫色
        
        if template_bbox is not None:
            # 计算预测框的左上角和右下角坐标
            template_bbox_x0, template_bbox_y0, template_bbox_x1, template_bbox_y1 = int(template_bbox[0]), int(template_bbox[1]), int(template_bbox[2]), int(template_bbox[3])
            if j == 0: 
                template_top_left = (template_bbox_x0, template_bbox_y0)
                template_bottom_right = (template_bbox_x0 + template_bbox_x1, template_bbox_y0 + template_bbox_y1)
                # 画预测框
                cv2.rectangle(img, template_top_left, template_bottom_right, (0, 0, 0), 2)  # black
            else:
                cv2.rectangle(img, [template_bbox_x0, template_bbox_y0], [template_bbox_x1, template_bbox_y1], (0, 0, 0), 2) # black

        if eye is not None:
            # 计算预测框的左上角和右下角坐标
            # eye_x, eye_y, eye_w, eye_h = int(eye[0]), int(eye[1]), int(eye[2]), int(eye[3])
            # eye_top_left = (eye_x - eye_w//2, eye_y - eye_h//2)
            # eye_bottom_right = (eye_x + eye_w//2, eye_y + eye_h//2)
            # 画预测框
            # cv2.rectangle(img, eye_top_left, eye_bottom_right, (0, 255, 0), 2)  # 绿色框
            # 标记预测值
            # text_org = (eye_top_left[0], eye_top_left[1] - 10)
            # cv2.putText(img, f'{eye_frame_auc_no_ext[s][j].item():.2f}', text_org, font, font_scale, (0, 255, 0), line_type)
            eye_x, eye_y= int(eye[0]), int(eye[1])
            cv2.drawMarker(img, (eye_x, eye_y), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2)  # 绿色"+"

        # 写入视频
        video.write(img)

    # 释放视频写入对象
    video.release()
    cv2.destroyAllWindows()

样本数量： 7


uav8: 100%|██████████| 301/301 [00:02<00:00, 121.96it/s]
